### Code for logistic matrix factorization 
Paper https://www.zora.uzh.ch/id/eprint/150303/1/two-class-cf.pdf

Converted code from theano: https://github.com/uzh/tccf/blob/master/softmax_mf_theano.py

In [233]:
import os 
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.autograd import Variable as V
from torch.utils.data import DataLoader
import torch.optim as optim

In [272]:
### Class for Logistic Model 
class LogisticMF(nn.Module):
    
    def __init__(self, num_ccs, num_items, num_factors, alpha):
        super(LogisticMF, self).__init__()
        
        ### Embeddings for the ccs and items
        self.ccs = nn.Embedding(num_ccs, num_factors)
        self.item = nn.Embedding(num_items, num_factors)
        ### bias for the ccs and items 
        self.ccs_bias = nn.Embedding(num_ccs, 1)
        self.item_bias = nn.Embedding(num_items, 1)      
        
        ### initialization of the weights
        self.ccs.weight.data.uniform_(-.01, .01)
        self.item.weight.data.uniform_(-.01, .01)
        self.ccs_bias.weight.data.uniform_(-.01, .01)
        self.item_bias.weight.data.uniform_(-.01, .01)
        
        ### regularization factor
        self.alpha = alpha

    def forward(self, pairs, target = None, loss_func = None):
        ### pairs is an Nx2 tensor that contains ccs, item pairs
        codes, features = pairs[:,0], pairs[:,1]
        c, it = self.ccs(codes), self.item(features)
        estimated_matrix = (c*it).sum(1)
        estimated_matrix_b = estimated_matrix + self.ccs_bias(codes).squeeze() + self.item_bias(features).squeeze()
        if target == None:
            return estimated_matrix_b
        else:
            loss = loss_func(estimated_matrix_b, target)
            l2_reg = self.alpha / 2 * (torch.norm(model.ccs.weight, p = 2) + torch.norm(model.item.weight, p = 2))
            loss += l2_reg
            return loss

### softmax matrix factorization
class S_MF(nn.Module):
    
    def __init__(self, num_ccs, num_items, num_factors, alpha):
        super(S_MF, self).__init__()

        ### Embeddings for the ccs and items
        self.ccs = nn.Embedding(num_ccs, num_factors)
        self.item_pos = nn.Embedding(num_items, num_factors)
        self.item_neg = nn.Embedding(num_items, num_factors)

        ### bias for the ccs and items 
        self.ccs_bias = nn.Embedding(num_ccs, 1)
        self.item_bias_pos = nn.Embedding(num_items, 1) 
        self.item_bias_neg = nn.Embedding(num_items, 1)      

        ### initialization of the weights
        self.ccs.weight.data.uniform_(-.01, .01)
        self.item_pos.weight.data.uniform_(-.01, .01)
        self.item_neg.weight.data.uniform_(-.01, .01)
        self.ccs_bias.weight.data.uniform_(-.01, .01)
        self.item_bias_pos.weight.data.uniform_(-.01, .01)
        self.item_bias_neg.weight.data.uniform_(-.01, .01)
        
        ### regularization factor
        self.alpha = alpha
    
    def forward(self, pairs, target = None):
        ### pairs is an Nx2 tensor that contains ccs, item pairs
        codes, features = pairs[:,0], pairs[:,1]
        c, it_pos, it_neg = self.ccs(codes), self.item_pos(features), self.item_neg(features)
        matrix_pos = (c*it_pos).sum(1)
        matrix_neg = (c*it_neg).sum(1)
        matrix_pos_b = matrix_pos + self.ccs_bias(codes).squeeze() + self.item_bias_pos(features).squeeze()
        matrix_neg_b = matrix_neg + self.ccs_bias(codes).squeeze() + self.item_bias_neg(features).squeeze()
        if target == None:
            return matrix_pos_b, matrix_neg_b    
        else:
            delta = nn.Threshold(0.5,0)
            pos = delta(target)
            neg = delta(-target)
            loss = (-pos * matrix_pos_b - neg * matrix_neg_b + torch.log(1 + torch.exp(matrix_pos_b) + torch.exp(matrix_neg_b))).sum()
            l2 = self.alpha / 3 * (torch.norm(model.ccs.weight, p = 2) + torch.norm(model.item_pos.weight, p = 2) + torch.norm(model.item_neg.weight, p = 2))
            loss = loss + l2 
            return loss

### loss functions for logistic matrix factorization
def loss_tc_mf1(output, target):
    delta = nn.Threshold(0.5, 0)
    pos = delta(target)
    loss = (-pos * output + torch.log(1 + torch.exp(output))).sum()
    return loss

def loss_tc_mf(output, target):
    delta = nn.Threshold(0.5, 0)
    pos = delta(target)
    neg = delta(-target)
    indicator = pos + neg
    loss = (-pos * output + indicator * torch.log(1 + torch.exp(output))).sum()
    return loss



In [288]:
### dump example to run the above models
n_epoch = 40
lr = 0.1
alpha = 0.0005 
training_loss = []

### should be 'smf' or 'logistic_mf'
used_model = 'logistic_mf'
### loss_tc_mf1 or loss_tc_mf. Used only if 'logstic_mf' model will be used. 
loss_func = loss_tc_mf
if loss_func not in [loss_tc_mf, loss_tc_mf1]:
    raise Exception("loss_func variable got {}, but it accepts only loss_tc_mf or loss_tc_mf1".format(loss_func))


# create model and optimizer
num_ccs = 4
num_items = 5
num_factors = 3


x = torch.tensor([[1, 0, -1, 0, 1], [1, 1, 0, 1, 1], [-1, -1, 0, 0 ,0], [1,1,-1,-1,-1]])
df = pd.DataFrame(data=x.numpy())
x_train = df.stack().reset_index().rename(columns={'level_0':'ccs_idx','level_1':'item_idx', 0:'rating'}).to_numpy()

if used_model == 'smf':
    model = S_MF(num_ccs, num_items, num_factors, alpha).cuda()
elif used_model == 'logistic_mf':
    model = LogisticMF(num_ccs, num_items, num_factors, alpha).cuda()
else:
    raise Exception("used_model variable got {}, but it accepts only smf or logistic_mf".format(used_model))
    
### weight decay is not used because the l2 regularization was used in the loss functions 
opt = optim.SGD(model.parameters(), lr, momentum=0.9)

train_loader = DataLoader(x_train, batch_size=2, shuffle=True)
for epoch in range(n_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        # get the inputs
        inputs = batch.long().cuda()
        true_val = V(batch[:, 2].float()).cuda()

        # zero the parameter gradients
        opt.zero_grad()

        # forward + backward + optimize
        if used_model == 'logistic_mf':
            loss = model.forward(inputs, true_val, loss_func)
        else: 
            loss = model.forward(inputs, true_val)
    
        loss.backward()
        opt.step()

        # print statistics
        running_loss += loss.item()
        if i % (len(train_loader)) == (len(train_loader) - 1):    # print every 2000 mini-batches
            training_loss.append((running_loss/len(train_loader)))
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss/len(train_loader)}")
            #epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
print('Finished Training')

[1, 10] loss: 0.9430566326350345
[2, 10] loss: 0.7025096127553297
[3, 10] loss: 0.45789282619953153
[4, 10] loss: 0.34755476713180544
[5, 10] loss: 0.2955535993100057
[6, 10] loss: 0.2511948686093092
[7, 10] loss: 0.22838901025243102
[8, 10] loss: 0.19627347010391533
[9, 10] loss: 0.17505496962112374
[10, 10] loss: 0.15911585781723261
[11, 10] loss: 0.1386752274062019
[12, 10] loss: 0.12007428063661792
[13, 10] loss: 0.10150305233546533
[14, 10] loss: 0.08129603624693119
[15, 10] loss: 0.0654752429574728
[16, 10] loss: 0.05254608520772308
[17, 10] loss: 0.04468314724508673
[18, 10] loss: 0.03632223354652524
[19, 10] loss: 0.031832817499525844
[20, 10] loss: 0.027891686710063368
[21, 10] loss: 0.02492203686852008
[22, 10] loss: 0.022577126056421547
[23, 10] loss: 0.021087916928809135
[24, 10] loss: 0.019579565222375094
[25, 10] loss: 0.01847477356204763
[26, 10] loss: 0.017607847461476922
[27, 10] loss: 0.016597243305295706
[28, 10] loss: 0.0158624772913754
[29, 10] loss: 0.015246757760

In [292]:
###check the estimated matrix comparing to the target one
print(x)
if used_model == 'smf':
    print(torch.matmul(model.ccs.weight, torch.transpose(model.item_pos.weight + model.item_neg.weight, 0, 1)))
elif used_model == 'logistic_mf':
    print(torch.matmul(model.ccs.weight, torch.transpose(model.item.weight, 0, 1)))
else:
    raise Exception("used_model variable got {}, but it accepts only smf or logistic_mf".format(used_model))

tensor([[ 1,  0, -1,  0,  1],
        [ 1,  1,  0,  1,  1],
        [-1, -1,  0,  0,  0],
        [ 1,  1, -1, -1, -1]])
tensor([[-1.3516, -1.8634, -0.6829,  1.5519,  2.4966],
        [-0.8808, -1.1887, -0.4041,  0.9894,  1.5676],
        [-1.5965, -2.1444, -0.7183,  1.7818,  2.8163],
        [ 3.2821,  4.4213,  1.4949, -3.6769, -5.8208]], device='cuda:0',
       grad_fn=<MmBackward>)
